In [7]:
## Import required libraries
import re
import pandas as pd
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession

In [8]:
main_url = "https://election.thestar.com.my/"
session = HTMLSession()
r = session.get(main_url)

## Previous testing individually
# r = session.get('https://election.thestar.com.my/ft.html')
# r = session.get('https://election.thestar.com.my/johor.html')
# r = session.get('https://election.thestar.com.my/kelantan.html')
# r = session.get('https://election.thestar.com.my/perak.html')

In [11]:
## scraping svg
## We need to use asynchronous function because map svg is rendered by js
async def get_svg(url: str,region):
    asession = AsyncHTMLSession()
    r = await asession.get(url)
    await r.html.arender(sleep=5)
    tab_content = r.html.find(".tab-content",first=True)
    tab_panels = tab_content.find(".tab-pane")
    print("Length of tab_panels: ",len(tab_panels))
    for a in range(len(tab_panels)):
        tab_attributes = tab_panels[a].attrs
        tab_id = tab_attributes.get("id")
        if tab_id == 'f1':
            tab_name = "parliament"
            svg_data = tab_panels[0].find("#svgP",first=True).html
            # print(type(svg_data))
            if svg_data != None:
                ## Convert string into byte 
                svg_byte = bytes(svg_data, encoding="utf-8")
                given_name = region + "_" + tab_name
                with open(f'./svgs/{given_name}.svg', 'wb+') as f:
                    f.write(svg_byte)
                print(f"{given_name} svg write!")
            else:
                print("Not found svg for Parliment")
            
        if tab_id =='f2':
            tab_name = "state"
            svg_data = tab_panels[1].find("#svgN", first=True).html
            if svg_data != None:
                ## Convert string into byte 
                svg_byte = bytes(svg_data, encoding="utf-8")
                given_name = region + "_" + tab_name
                with open(f'./svgs/{given_name}.svg', 'wb+') as f:
                    f.write(svg_byte)
                print(f"{given_name} svg write!")
            else:
                print("Not found svgN for State")
    await asession.close()
    # return svg_data.html

In [ ]:
'''
## testing with one url
test_url = "https://election.thestar.com.my/johor.html"
await get_svg(test_url,'Johor')
'''
'''
## Testing block to convert string to bytes
#convert string to byte
svg_byte = bytes(data_web, "utf-8")
with open('test.svg','wb+') as f:
    f.write(svg_byte)
'''

In [12]:
## Scraping data for looping webpages
state = r.html.find("ul.state-list", first=True)
state_list = state.find("a")
for i in range(len(state_list)):
    list_attrs = state_list[i].attrs
    # print(list_attrs)
    urls = main_url + list_attrs.get('href')
    state_name = state_list[i].text.replace(" ","")
    print(state_name,"|",urls)
    new_session = HTMLSession()
    ns = new_session.get(urls)
    # print(ns.url)
    # print(ns.status_code)
    if ns.status_code == 200:
        ## saving svg files for each parliment & state map
        await get_svg(urls,state_name)
        print(f"Scraping SVG for {state_name} done!")
        
        ## scraping results & facts(demographic) data
        # scrape_panels(ns, state_name)
    else:
        print("Connection error!")

FederalTerritories | https://election.thestar.com.my/ft.html
Length of tab_panels:  1
<class 'str'>
FederalTerritories_parliament svg write!
Scraping SVG for FederalTerritories done!
Johor | https://election.thestar.com.my/johor.html
Length of tab_panels:  2
<class 'str'>
Johor_parliament svg write!
Johor_state svg write!
Scraping SVG for Johor done!
Kedah | https://election.thestar.com.my/kedah.html
Length of tab_panels:  2
<class 'str'>
Kedah_parliament svg write!
Kedah_state svg write!
Scraping SVG for Kedah done!
Kelantan | https://election.thestar.com.my/kelantan.html
Length of tab_panels:  2
<class 'str'>
Kelantan_parliament svg write!
Kelantan_state svg write!
Scraping SVG for Kelantan done!
Melaka | https://election.thestar.com.my/malacca.html
Length of tab_panels:  2
<class 'str'>
Melaka_parliament svg write!
Melaka_state svg write!
Scraping SVG for Melaka done!
NegriSembilan | https://election.thestar.com.my/negerisembilan.html
Length of tab_panels:  2
<class 'str'>
NegriSemb

AttributeError: 'NoneType' object has no attribute 'html'

In [109]:
### Scraping function to get results & demographic data
def scrape_panels(conn, region):
    tab_content = conn.html.find(".tab-content")
    
    ## Only ft & sarawak pages will have one tab_panel , others length will be 2
    tab_panels = tab_content[0].find(".tab-pane")
    # print("Tab Panels (P or S):{}".format(len(tab_panels)))
    for a in range(len(tab_panels)):
        ## Divide parliament or state tabs
        tab_attributes = tab_panels[a].attrs
        tab_id = tab_attributes.get("id")
        if tab_id == 'f1':
            tab_name = "parliament"
            
            ## Getting Parliament > Result , Parliament > Facts(demographic)
            p_result,p_fact = scrape_data(tab_panels[0],tab_name)
            p_result['state_region'] = region
            p_fact['state_region'] = region
            
            ## should I export csv under this function?
            ## Export with custom name
            given_name = region + "_" + tab_name
            p_result.to_csv(f'./data/{given_name}_result.csv')
            p_fact.to_csv(f'./data/{given_name}_fact.csv')
            
        if tab_id == 'f2':
            tab_name = "state"
       
            ## Getting State > Result , State > Facts(demographic)
            s_result,s_fact = scrape_data(tab_panels[1],tab_name)
            s_result['state_region'] = region
            s_fact['state_region'] = region
            
            ## Export with custom name
            given_name = region + "_" + tab_name
            s_result.to_csv(f'./data/{given_name}_result.csv')
            s_fact.to_csv(f'./data/{given_name}_fact.csv')
        # print(tab_name)
        # print(f"Tab-Panel-{a}")
        # print("---xxx---")
        
    ## If I exported CSVs , I don't need to return dfs.
    if len(tab_panels) == 1:
        ## Returning parliament result & facts(demographic) data
        print("Parliament only data")
        # return p_result,p_fact
    elif len(tab_panels) == 2:
        ## Returning both perliment+state result & facts(demographic) data
        print("Parliament + State data")
        # return p_result,p_fact,s_result,s_fact
    else:
        print("No Data found!")

In [81]:
def scrape_data(tab_panel,tab_name):
    ## scrape data under .winner-list > multiple .panel
    panels = tab_panel.find(".panel")
    # print(len(panels))
    result_data = []
    fact_data = []
        
    for i in range(len(panels)):
        ## Inside .panel-heading
        header = panels[i].find("h4.panel-title")[0].text.split(" ")
        pcode = header[0]
        pname = ' '.join(header[1:])
        print(pcode,"|",pname)
        
        ## Inside .panel-body 
        wrapper = panels[i].find(".progress-wrapper")
        # print(len(wrapper))
        # print("---xx---")
        
        for j in range(len(wrapper)):
            results_by_territories = {}
            name = wrapper[j].find("p.name-candidate")[0].text
            # print(name)
            number_of_voters = wrapper[j].find(".number-of-voters")[0].text
            # print(number_of_voters)
            results_by_territories["name"] = name
            results_by_territories["number_of_voters"] = number_of_voters
            
            ## Adding pcode,pname from .panel-heading
            results_by_territories["panel_code"]= pcode
            results_by_territories["panel_name"]= pname
            
            ## Adding tab name (Parliament or State)
            results_by_territories["tab"] = tab_name
            result_data.append(results_by_territories)
        
        
        blue_box = panels[i].find(".blue-box-info")
        
        facts = {}
        if blue_box != None:
            # print("Found Blue_box!")
            ## Do I need to find "voters-vote" class?
            info = blue_box[0].find("p")
            num_regex = re.compile(r'[\d,\.\s\(%\)]+')
            registered_voters = ''.join(num_regex.findall(info[0].text)).strip()
            majority = ''.join(num_regex.findall(info[1].text)).strip()
            spoilt_votes = ''.join(num_regex.findall(info[2].text)).strip()
            unreturned_votes = ''.join(num_regex.findall(info[3].text)).strip()
            voter_turnout = ''.join(num_regex.findall(info[4].text)).strip()
            demographics = info[5].text
            # print(registered_voters,"|",majority,"|",spoilt_votes,"|",unreturned_votes,"|",voter_turnout)
            # print(demographics)
    
            facts['registered_voters'] = registered_voters
            facts['majority'] = majority
            facts['spoilt_votes'] = spoilt_votes
            facts['unreturned_votes']= unreturned_votes
            facts['voter_turnout'] = voter_turnout
            facts['demographics']= demographics
            
            ## Adding pcode,pname from .panel-heading
            facts['panel_code'] = pcode
            facts['panel_name'] = pname
            
            ## Adding tab names (Parliament or State)
            facts['tab'] = tab_name
            fact_data.append(facts)
            
            # print(len(info))
        # print(results_by_territories)
    # print(result_data)
    result_df = pd.DataFrame(result_data)
    fact_df = pd.DataFrame(fact_data)
    # print(result_df.shape)
    # print(result_df.columns)
    # print(result_df.head())
    return result_df,fact_df